In [ ]:
import pandas as pd

In [ ]:
# Extraction des SIRET des DECP
decp_titulaires_sirets_df = pd.read_parquet(
    "dist/decp.parquet", columns=["titulaire.id", "titulaire.typeIdentifiant"]
)

decp_titulaires_sirets_df = decp_titulaires_sirets_df[
    decp_titulaires_sirets_df["titulaire.typeIdentifiant"] == "SIRET"
]
decp_titulaires_sirets_df = decp_titulaires_sirets_df.drop_duplicates()
decp_titulaires_sirets_df.index.size

In [ ]:
# Récupération des données SIRET titulaires
dtypes_etablissements = {
    "siret": "object",
    "siren": "object",
    "longitude": "float",
    "latitude": "float",
    "activitePrincipaleEtablissement": "object",
    "codeCommuneEtablissement": "object",
    "etatAdministratifEtablissement": "category",
}
etablissement_df_chunked = pd.read_csv(
    "data/StockEtablissement_utf8_geo.csv.gz",
    chunksize=(500000),
    dtype=dtypes_etablissements,
    index_col=None,
    usecols=[
        "siret",
        "siren",
        "longitude",
        "latitude",
        "activitePrincipaleEtablissement",
        "codeCommuneEtablissement",
        "etatAdministratifEtablissement",
    ],
)

merged_chunks_list = []

# 6 min 20 en gardant tous les merges, 10 000 chunks. Resultat 38M rows!
# 6 min 27 avec inner merge et en ne gardant que les merge non vides
# 4 min 04 avec 200 000 chunks
# 3 min 27 avec 500 000 chunks
# 3 min 05 avec 1M chunks

with etablissement_df_chunked as reader:
    for df_chunk in reader:
        merge = pd.merge(
            decp_titulaires_sirets_df,
            df_chunk,
            how="inner",
            left_on="titulaire.id",
            right_on="siret",
        )
        if merge.index.size > 0:
            merged_chunks_list.append(merge)

decp_titulaires_sirets_df = pd.concat(merged_chunks_list).drop(columns=["siret"])

del etablissement_df_chunked, df_chunk

decp_titulaires_sirets_df

In [ ]:
# Récupération des données SIREN
# Possibilité d'utiliser les URL sources en prod au lieu d'un fichier local

dtypes_uniteLegales = {
    "siren": "object",
    "categorieEntreprise": "object",  # doit être object, car il y a des NaN
    "etatAdministratifUniteLegale": "category",
    "economieSocialeSolidaireUniteLegale": "object",  # doit être object, car il y a des NaN
    "categorieJuridiqueUniteLegale": "object",  # object plutôt que catégorie pour faire des modifications plus tard
}
unite_legale_df_chunked = pd.read_csv(
    "./data/StockUniteLegale_utf8.zip",
    index_col=None,
    dtype=dtypes_uniteLegales,
    sep=",",
    chunksize=500000,
    usecols=[
        "siren",
        "categorieEntreprise",
        "etatAdministratifUniteLegale",
        "economieSocialeSolidaireUniteLegale",
        "categorieJuridiqueUniteLegale",
    ],
)

merged_chunks_list = []

with unite_legale_df_chunked as reader:
    for df_chunk in reader:
        merge = pd.merge(decp_titulaires_sirets_df, df_chunk, how="inner", on="siren")
        if not merge.empty and merge.notnull().any().any() and len(merge) >= 1:
            merged_chunks_list.append(merge)
del unite_legale_df_chunked, df_chunk

decp_titulaires_sirets_df = pd.concat(merged_chunks_list)

del merged_chunks_list

In [ ]:
# Raccourcissement du code commune
decp_titulaires_sirets_df["departement"] = decp_titulaires_sirets_df[
    "codeCommuneEtablissement"
].str[:2]
decp_titulaires_sirets_df = decp_titulaires_sirets_df.drop(
    columns=["codeCommuneEtablissement"]
)

# # Raccourcissement de l'activité principale
# pas sûr de pourquoi je voulais raccourcir le code NAF/APE. Pour récupérérer des libellés ?
# decp_titulaires_sirets_df['activitePrincipaleEtablissement'] = decp_titulaires_sirets_df['activitePrincipaleEtablissement'].str[:-3]

# Correction des données ESS et état
decp_titulaires_sirets_df["etatAdministratifUniteLegale"] = decp_titulaires_sirets_df[
    "etatAdministratifUniteLegale"
].cat.rename_categories({"A": "Active", "C": "Cessée"})
decp_titulaires_sirets_df[
    "economieSocialeSolidaireUniteLegale"
] = decp_titulaires_sirets_df["economieSocialeSolidaireUniteLegale"].replace(
    {"O": "Oui", "N": "Non"}
)

In [ ]:
# Récupération et raccourcissement des categories juridiques du fichier SIREN
decp_titulaires_sirets_df["categorieJuridiqueUniteLegale"] = (
    decp_titulaires_sirets_df["categorieJuridiqueUniteLegale"].astype(str).str[:2]
)

# Récupération des libellés des catégories juridiques
cj_df = pd.read_csv("data/cj.csv", index_col=None, dtype="object")
decp_titulaires_sirets_df = pd.merge(
    decp_titulaires_sirets_df,
    cj_df,
    how="left",
    left_on="categorieJuridiqueUniteLegale",
    right_on="Code",
)
decp_titulaires_sirets_df["categorieJuridique"] = decp_titulaires_sirets_df["Libellé"]
decp_titulaires_sirets_df = decp_titulaires_sirets_df.drop(
    columns=["Code", "categorieJuridiqueUniteLegale", "Libellé"]
)
decp_titulaires_sirets_df

In [ ]:
# Renommage des colonnes

renaming = {
    "activitePrincipaleEtablissement": "codeAPE",
    "etatAdministratifUniteLegale": "etatEntreprise",
    "etatAdministratifEtablissement": "etatEtablissement",
}

decp_titulaires_sirets_df = decp_titulaires_sirets_df.rename(columns=renaming)


# Jointure avec les données brutes

final_columns = [
    "id",
    "uid",
    "acheteur.id",
    "acheteur.nom",
    "nature",
    "objet",
    "codeCPV",
    "lieuExecution.code",
    "lieuExecution.typeCode",
    "lieuExecution.nom",
    "dureeMois",
    "dateNotification",
    "montant",
    "titulaire.id",
    "titulaire.typeIdentifiant",
    "titulaire.denominationSociale",
    "codeAPE",
    "departement",
    "categorieEntreprise",  # plutôt que categorie
    "categorieJuridique",  # libellé
    # "categorieJuridiqueLibelle1",
    # "categorieJuridiqueLibelle2",
    "etatEtablissement",
    "etatEntreprise",
    "longitude",
    "latitude",
    "donneesActuelles",
    "anomalies",
]

decp_df = pd.read_parquet("dist/decp.parquet")
decp_titulaires_df = pd.merge(
    decp_df,
    decp_titulaires_sirets_df,
    on=["titulaire.id", "titulaire.typeIdentifiant"],
    how="left",
)
decp_titulaires_df = decp_titulaires_df[final_columns]
del decp_df, decp_titulaires_sirets_df

In [ ]:
# Enregistrement

decp_titulaires_df.to_csv("dist/decp-titulaires.csv", index=None)
decp_titulaires_df.to_parquet("dist/decp-titulaires.parquet", index=None)